In [1]:
# imports

import sqlalchemy
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.preprocessing import RobustScaler

import pickle
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor


In [2]:
#Exportamos datasets(elegir esta opción )

diamonds_train = pd.read_csv('./data/diamondssql.csv')
diamonds_test = pd.read_csv('./data/diamonds_test.csv')

In [3]:
#Quitamos la columna city del dataset: 
diamonds_train = diamonds_train.drop(['city'], axis=1)

In [4]:
## A continuación vamos a cambiar los datos de tipo de color en valores numericos con la función encoder:
from sklearn.preprocessing import OrdinalEncoder
# Creamos el codificador indicandole el orden de la variables
encoder = OrdinalEncoder(categories=[['J', 'I', 'H', 'G', 'F', 'E', 'D']])
# Ajustamos el codificador con la variable education y la transformamos
encoder.fit(diamonds_train[["color"]])
diamonds_train["color-encoded"] = encoder.transform(diamonds_train[["color"]])
diamonds_train.drop("color", axis = 1 , inplace = True)

# Ajustamos el codificador con la variable education y la transformamos
encoder.fit(diamonds_test[["color"]])
diamonds_test["color-encoded"] = encoder.transform(diamonds_test[["color"]])
diamonds_test.drop("color", axis = 1 , inplace = True)

In [5]:
# Creamos el codificador indicandole el orden de la variables de CUT
encoder = OrdinalEncoder(categories=[['Premium', 'Very Good', 'Fair', 'Good', 'Ideal']])
# Ajustamos el codificador con la variable education y la transformamos
encoder.fit(diamonds_train[["cut"]])
diamonds_train["cut-encoded"] = encoder.transform(diamonds_train[["cut"]])
diamonds_train.drop("cut", axis = 1 , inplace = True)
# Ajustamos el codificador con la variable education y la transformamos
encoder.fit(diamonds_test[["cut"]])
diamonds_test["cut-encoded"] = encoder.transform(diamonds_test[["cut"]])
diamonds_test.drop("cut", axis = 1 , inplace = True)

In [6]:
# Creamos el codificador indicandole el orden de la variables de Clarity
encoder = OrdinalEncoder(categories=[['VS2', 'SI1', 'SI2', 'VS1', 'IF', 'VVS1', 'VVS2', 'I1']])
# Ajustamos el codificador con la variable education y la transformamos
encoder.fit(diamonds_train[["clarity"]])
diamonds_train["clarity-encoded"] = encoder.transform(diamonds_train[["clarity"]])
diamonds_train.drop("clarity", axis = 1 , inplace = True)
# Ajustamos el codificador con la variable education y la transformamos
encoder.fit(diamonds_test[["clarity"]])
diamonds_test["clarity-encoded"] = encoder.transform(diamonds_test[["clarity"]])
diamonds_test.drop("clarity", axis = 1 , inplace = True)

In [7]:
#Definimos la columna target y configuramos las features del dataset: categóricas o numéricas

target = "price"
features = ['depth', 'carat', 'table', 'x', 'y', 'z', 'color-encoded', 'cut-encoded', 'clarity-encoded']

In [8]:
# Scaler a train db

scaler = RobustScaler()
x =scaler.fit_transform(diamonds_train[features])
y = diamonds_train[target]



In [9]:
#divido el dataset en 80/20 para poder realizar testing
X_train, X_validation, y_train, y_validation=train_test_split(x, y, test_size=0.2, random_state=42)


In [10]:
model1 = LinearRegression()
model2 = RandomForestRegressor(random_state=42)


# Entrenamos el modelo con en el dataset de train
model2.fit(X_train,y_train)

y_pred_train = model2.predict(X_validation)

In [11]:
##Sacar RMS de tu modelo de train bd

RMSE_ERROR_sklearn = mean_squared_error(y_validation, y_pred_train, squared = False)
print(RMSE_ERROR_sklearn)

551.1019282650497


In [12]:
# Scaler a Test db

scaler = RobustScaler()
scaler_test =scaler.fit_transform(diamonds_test[features])


In [13]:
y_pred_test = model2.predict(scaler_test).clip(0,30000)


In [14]:
submission_test =pd.DataFrame({'id': diamonds_test['id'],'price': y_pred_test})

submission_test.head()

,id,price
0,0,2881.62
1,1,5355.66
2,2,9589.35
3,3,4173.35
4,4,1680.62


In [15]:
submission_test.to_csv('submission_test_randomforest11.csv', index =False)